In [ ]:
import keras
from keras import models
from keras.preprocessing import image
import tensorflow as tf
import numpy as np
from keras import backend as K
from keras.applications import vgg16
import matplotlib.pyplot as plt
%matplotlib inline

tf.logging.set_verbosity(tf.logging.ERROR)

model = vgg16.VGG16(weights='imagenet',
              include_top=False)
model.summary()

In [ ]:
# Load a source image
img = image.load_img("/tmp/workspace/Pictures/cat_1700.jpg", target_size=(150, 150))
plt.imshow(img)
plt.show()

img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis=0)
img_tensor = vgg16.preprocess_input(img_tensor)

In [ ]:
# Set here the layer name and filter number for analysis
layer_name = 'block1_conv1'
filter_num = 0

In [ ]:
# Display raw kernel values of a filter in a convolutional layer
kernel = model.get_layer(layer_name).kernel
c_filter = kernel[:,:,:,filter_num]
print("Shape of filter No. {} in layer {}: {}".format(filter_num, layer_name, c_filter.shape))
print()
print("Raw kernel values:")
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(c_filter))

In [ ]:
# Display the activation of a single filter in a specific convolutional layer
layer_output = [model.get_layer(layer_name).output]

# Creates a model that will return these outputs, given the model input:
activation_model = models.Model(inputs=model.input, outputs=layer_output)

activation = activation_model.predict(img_tensor)
print("Activation map shape:", activation.shape)
plt.matshow(activation[0, :, :, filter_num])
plt.show()

In [ ]:
# Create a map of all filter activations in a specific layer
images_per_row = 16

n_features = activation.shape[-1]
size = activation.shape[1]
n_cols = n_features // images_per_row
display_grid = np.zeros((size * n_cols, images_per_row * size))
for col in range(n_cols):
    for row in range(images_per_row):
        channel_image = activation[0, :, :, col * images_per_row + row]
        # Post-process the feature to make it visually palatable
        channel_image -= channel_image.mean()
        channel_image /= (channel_image.std() + 1e-5)
        channel_image *= 64
        channel_image += 128
        channel_image = np.clip(channel_image, 0, 255).astype('uint8')
        display_grid[col * size : (col + 1) * size,
                     row * size : (row + 1) * size] = channel_image

scale = 1. / size
plt.figure(figsize=(scale * display_grid.shape[1],
                    scale * display_grid.shape[0]))
plt.title(layer_name)
plt.grid(False)
plt.imshow(display_grid, aspect='auto')
plt.show()